In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create database connection URL (for MySQL; change driver if needed)
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(db_url)

# Helper function to load a table into Pandas
def load_table(table_name: str) -> pd.DataFrame:
    """Load entire SQL table into a Pandas DataFrame."""
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql(query, engine)

# Load your tables
users_df = load_table("users")
activity_log_df = load_table("activity_log")
loan_applications_df = load_table("loan_applications")
loans_df = load_table("loans")

# Define constants
TODAY = pd.Timestamp.today().normalize()
LAST_MONTH_DATE = TODAY - pd.DateOffset(months=1)
BASIC_SALARY = 17000.00

Users:
    id   user_name             name surname        full_name  \
0   51        None         los_team    None             None   
1   78       admin   Fateh Alrabeai    None   Fateh Alrabeai   
2  280        None          LOS_DEV    None             None   
3  454  receivable       recievalbe    None       recievalbe   
4  455     finance  Finance Manager    None  Finance Manager   

              phone fcm_token  \
0       13415834745      None   
1  +20 10 3456 7890      None   
2       12697661167      None   
3  +20 10 3456 7890      None   
4        4444333322      None   

                                               email   age email_verified_at  \
0  1|zyI21DeCQ4OYoAbC|zOBvxZO8LgSZ74JSxA==|aG1iYy...  None              None   
1  1|fQ3Utl7Vcsym8jTC|hVeqO5bDD+R4swxCtABb|mcbPr5...  None              None   
2  1|aOVvEcsYQRWCbdmy|wyEPMwXjgQRcn85b67IXuw==|4T...  None              None   
3  1|BZQexJI4vLXB4Uix|vI6NFQF9JNA2zWrbJ3WDWV5mwrC...  None              None   
4  1|9AbJ

In [4]:
loan_applications_df

,id,loan_amount,tenure_months,purpose,justification_by_ai,justification_by_user,rejection_reason,missing_documents_note,step,loan_app_number,...,is_business_request_created,is_contract_signed,justified_by,created_at,updated_at,deleted_at,external_id,loan_amount_hash,tenure_months_hash,loan_app_number_hash
0,1,None,None,None,None,bvnv,None,None,1,1,...,0,0,78.0,2025-08-26 14:20:26,2025-08-26 15:01:49,None,08dde492-4b74-4284-84f5-f83b2e80e72c,None,None,None
1,2,None,None,None,None,None,None,None,1,LA-265902,...,0,0,NaN,2025-08-26 14:50:49,2025-08-26 14:53:28,None,e4c1v4f7-hhthth-025252-dvdvdv-vfbfbf,None,None,None
2,3,50000,None,None,None,test,None,None,1,1,...,0,0,78.0,2025-08-26 15:09:42,2025-08-26 15:26:14,None,08dde499-53d6-4694-8aa7-8f8c6ceed01f,enc_fa5c3bbfecc9c062e8aeaa575fcec4f800c2c964b8...,None,None
3,4,None,None,None,None,None,None,None,1,1,...,0,0,NaN,2025-08-26 15:49:13,2025-08-26 15:49:15,None,08dde49e-e4a0-48dd-891f-76a0456a50fe,None,None,None
4,5,100000,None,None,None,000,None,None,1,1,...,0,1,78.0,2025-08-26 16:09:01,2025-08-26 16:21:32,None,08dde4a1-9c56-446c-811d-2ced26963cbd,enc_846c44aaba99c432a74be298f0ac6a8904879b306f...,None,None
5,6,500000,None,None,None,test,None,None,1,1,...,0,1,78.0,2025-08-26 20:22:56,2025-08-26 20:42:47,None,08dde4c5-0ce4-4aaf-8630-1e45ac76bcfd,enc_5daf29921c67e5bdca5a549159f95bac3e70949697...,None,None
6,7,150000,None,None,None,111,None,None,1,1,...,0,1,78.0,2025-08-27 12:02:35,2025-08-27 12:48:06,None,08dde547-d0bc-4bf4-8ebd-d85cc263eb8c,enc_ceb7208941e1e78f4f7012f711e6557840b1257ae9...,None,None
7,8,None,None,None,None,None,None,None,1,1,...,0,0,NaN,2025-08-27 15:41:51,2025-08-27 15:46:54,None,08dde566-fcfb-483e-8db9-7d0951b30cbd,None,None,None
8,9,56000,None,None,None,hh,None,None,1,1,...,0,1,78.0,2025-08-31 05:10:24,2025-08-31 05:29:14,None,08dde833-7232-438f-8393-e548c78986bb,enc_65679550d8cafd3542c34d30b97c3d7b0eaf1e1efa...,None,None
9,10,None,None,None,None,None,None,None,1,1,...,0,0,NaN,2025-08-31 19:39:02,2025-08-31 19:42:47,None,08dde8ac-5d2c-44ce-848c-d4323c9a3d5a,None,None,None


In [2]:
# Filter loan applications: approved + within last month
approved_loans = loan_applications_df[
    (loan_applications_df['status'] == 'APPROVED') &
    (loan_applications_df['created_at'] >= LAST_MONTH_DATE)
]

# Join activity_log → loan_applications
activity_with_loans = activity_log_df.merge(
    approved_loans,
    left_on='subject_id',
    right_on='id',
    how='left',
    suffixes=('_al', '_loan')
)

# Filter only activity logs tied to loan applications
activity_with_loans = activity_with_loans[
    activity_with_loans['subject_type'] == 'App\\Models\\LoanApplication'
]

# Count loans issued per RO
ro_data = (
    users_df.rename(columns={'id': 'officer_id'})
    .merge(
        activity_with_loans,
        left_on='officer_id',
        right_on='causer_id',
        how='left'
    )
    .groupby(['officer_id', 'city_id'], as_index=False)
    .agg(loans_issued=('id_loan', 'count'))
)

# Only keep active ROs
ro_data = ro_data.merge(
    users_df[['id', 'status']],
    left_on='officer_id',
    right_on='id',
    how='left'
)
ro_data = ro_data[ro_data['status'] == 'ACTIVE'].drop(columns='status')


In [3]:
ro_data

,officer_id,city_id,loans_issued,id


In [ ]:
# Join supervisors to ro_data by city
team = users_df.merge(
    ro_data,
    left_on='city_id',
    right_on='city_id',
    how='left',
    suffixes=('_supervisor', '_ro')
)

# Count total ROs per supervisor
total_ros = team.groupby('id_supervisor')['officer_id'].nunique().reset_index(name='total_ros')

# Count achieving ROs (loans_issued > 0)
achieving_ros = team[team['loans_issued'] > 0].groupby('id_supervisor')['officer_id'].nunique().reset_index(name='achieving_ros')

# Merge counts
team_performance = users_df.rename(columns={'id': 'supervisor_id', 'full_name': 'supervisor_name'})
team_performance = team_performance.merge(total_ros, left_on='supervisor_id', right_on='id_supervisor', how='left').drop(columns='id_supervisor')
team_performance = team_performance.merge(achieving_ros, left_on='supervisor_id', right_on='id_supervisor', how='left').drop(columns='id_supervisor')

# Fill missing with 0
team_performance['total_ros'] = team_performance['total_ros'].fillna(0)
team_performance['achieving_ros'] = team_performance['achieving_ros'].fillna(0)

# Compute achievement percentage
team_performance['achievement_percentage'] = np.where(
    team_performance['total_ros'] > 0,
    round((team_performance['achieving_ros'] * 100.0) / team_performance['total_ros'], 2),
    0
)

# Compute base commission
team_performance['base_commission'] = team_performance['achieving_ros'] * 800

# Compute multiplier based on achievement
def get_multiplier(pct):
    if pct >= 90:
        return 1.20
    elif pct >= 80:
        return 1.00
    elif pct >= 70:
        return 0.80
    else:
        return 0.00

team_performance['multiplier'] = team_performance['achievement_percentage'].apply(get_multiplier)

# Compute final commission
team_performance['final_commission'] = round(
    team_performance['base_commission'] * team_performance['multiplier'], 2
)

# Add year/month columns based on last month
team_performance['year'] = LAST_MONTH_DATE.year
team_performance['month'] = LAST_MONTH_DATE.month


In [ ]:
# Join ROs to supervisors via city
par_data = users_df.rename(columns={'id': 'supervisor_id'}).merge(
    users_df.rename(columns={'id': 'ro_id'}),
    left_on='city_id',
    right_on='city_id',
    suffixes=('_supervisor', '_ro')
)

# Join ROs → loan applications → loans
par_data = par_data.merge(loan_applications_df, left_on='ro_id', right_on='user_id', how='inner')
par_data = par_data.merge(loans_df, left_on='id', right_on='loan_application_id', how='inner')

# Keep only active loans with outstanding balances
par_data = par_data[(par_data['status'] == 'ACTIVE') & (par_data['outstanding_balance'] > 0)]

# Compute PAR 30+
par_grouped = par_data.groupby('supervisor_id').apply(
    lambda df: round(
        (df.loc[(TODAY - df['due_date']).dt.days > 30, 'outstanding_balance'].sum() * 100.0)
        / df['outstanding_balance'].sum(), 2
    )
).reset_index(name='par_30_percentage')


In [ ]:
# Merge team performance with par data
final_df = team_performance.merge(par_grouped, on='supervisor_id', how='left')
final_df['par_30_percentage'] = final_df['par_30_percentage'].fillna(0)

# Apply P@R penalty
final_df['final_commission_after_par'] = np.where(
    final_df['par_30_percentage'] > 3.0,
    final_df['final_commission'] * (1 - (final_df['par_30_percentage'] - 3.0) / 100.0),
    final_df['final_commission']
)

# Add basic salary
final_df['basic_salary'] = BASIC_SALARY

# Total compensation
final_df['total_compensation'] = round(final_df['basic_salary'] + final_df['final_commission_after_par'], 2)

# Sort by highest compensation
final_df = final_df.sort_values(by='total_compensation', ascending=False).reset_index(drop=True)


In [ ]:
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")
print(final_df.head())
